In [1]:
import os, sys, glob, abc
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt, colors
from multiprocessing import Pool
import mla
from mla.threeml import spectral
import imp
import astromodels
import threeML
import pickle
from functools import partial
from mla.threeml.IceCubeLike import IceCubeLike
from threeML import *
import numpy.lib.recfunctions as rf
def read(filelist):
    data = []

    for f in sorted(filelist):
        x = np.load(f)
        if len(data) == 0: data = x.copy()
        else: data = np.concatenate([data, x])

    try:
        data=rf.append_fields(data, 'sindec',
                              np.sin(data['dec']),
                              usemask=False)
    except:
        pass
    return data


[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available
[INFO    ] Starting 3ML!
[WARNING ] no display variable set. using backend for graphics without display (agg)
[INFO    ] Starting 3ML!
[WARNING ] no display variable set. using backend for graphics without display (agg)
[WARNING ] ROOT minimizer not available
[WARNING ] Multinest minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software ins

In [2]:
#Loading all the data , MC and good run list
DATA_PATH = "/data/i3store/users/mjlarson/ps_tracks/version-004-p00"

# Read in all of the MC files 
sim_files = DATA_PATH + "/IC86*MC*npy"
sim = np.load([i for i in glob.glob(sim_files) if "2011" not in i][0])
n_keep = int(0.5*len(sim)) #server don't have that much memory for me to 
sim = np.random.choice(sim, n_keep)
sim=rf.append_fields(sim, 'sindec',
                     np.sin(sim['dec']),
                     usemask=False)


# Read in all of the data files
data_files = DATA_PATH + "/IC86_*exp.npy"
listofdata = []
data = read([i for i in glob.glob(data_files)])


# Set the angular error floor to 0.2 degrees
#data['angErr'][data['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
sim['angErr'][sim['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
np.random.seed(2) #for reproduce
data['ra'] = np.random.uniform(0, 2*np.pi, size=len(data))
grlfile = DATA_PATH + "/GRL/IC86*_exp.npy"
grl = read([i for i in glob.glob(grlfile)])
livetime = np.sum(grl['livetime'])
bkg_days = grl['stop'][-1]-grl['start'][0]
background_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
inject_signal_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
if 'sindec' not in data.dtype.names:
    data = rf.append_fields(
        data,
        'sindec',
        np.sin(data['dec']),
        usemask=False,
    )
if 'sindec' not in sim.dtype.names:
    sim = rf.append_fields(
        sim,
        'sindec',
        np.sin(sim['dec']),
        usemask=False,
    )
config = mla.generate_default_config([
    mla.threeml.data_handlers.ThreeMLDataHandler,
    mla.PointSource,
    mla.SpatialTermFactory,
    mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory,
    mla.TimeTermFactory,
    mla.LLHTestStatisticFactory
])
config['PointSource']['name'] = 'TXS0506+056'
config['PointSource']['ra'] = mla.ra_to_rad(5, 9, 25.9645434784)
config['PointSource']['dec'] = mla.dec_to_rad(1, 5, 41, 35.333636817)
config['ThreeMLDataHandler']['dec_bandwidth (rad)'] = np.deg2rad(3)
config['ThreeMLDataHandler']['dec_cut_location']=mla.dec_to_rad(1, 5, 41, 35.333636817)
source = mla.PointSource(config['PointSource'])

data_handler = mla.threeml.data_handlers.ThreeMLDataHandler(
    config['ThreeMLDataHandler'], sim, (data, grl),background_time_profile=background_time_profile,signal_time_profile=inject_signal_time_profile)
spatial_term_factory = mla.SpatialTermFactory(config['SpatialTermFactory'], data_handler, source)
energy_term_factory = mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory(config['ThreeMLPSEnergyTermFactory'], data_handler,source)
time_term_factory = mla.TimeTermFactory(config['TimeTermFactory'],background_time_profile,inject_signal_time_profile)
llh_factory = mla.LLHTestStatisticFactory(config['LLHTestStatisticFactory'],[spatial_term_factory,energy_term_factory,time_term_factory])
icecube=IceCubeLike("txs",data,data_handler,llh_factory,source,verbose=False)


WARNING RuntimeWarning: invalid value encountered in true_divide



In [3]:
#Loading all the data , MC and good run list
DATA_PATH = "/data/i3store/users/mjlarson/ps_tracks/version-004-p00"

# Read in all of the MC files 
sim_files = DATA_PATH + "/IC79*MC*npy"
sim = np.load([i for i in glob.glob(sim_files)][0])
#n_keep = int(0.5*len(sim)) #server don't have that much memory for me to 
#sim = np.random.choice(sim, n_keep)
sim=rf.append_fields(sim, 'sindec',
                     np.sin(sim['dec']),
                     usemask=False)


# Read in all of the data files
data_files = DATA_PATH + "/IC79_*exp.npy"
listofdata = []
data = read([i for i in glob.glob(data_files)])


# Set the angular error floor to 0.2 degrees
#data['angErr'][data['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
sim['angErr'][sim['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
np.random.seed(2) #for reproduce
data['ra'] = np.random.uniform(0, 2*np.pi, size=len(data))
grlfile = DATA_PATH + "/GRL/IC79*_exp.npy"
grl = read([i for i in glob.glob(grlfile)])
livetime = np.sum(grl['livetime'])
bkg_days = grl['stop'][-1]-grl['start'][0]
background_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
inject_signal_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
if 'sindec' not in data.dtype.names:
    data = rf.append_fields(
        data,
        'sindec',
        np.sin(data['dec']),
        usemask=False,
    )
if 'sindec' not in sim.dtype.names:
    sim = rf.append_fields(
        sim,
        'sindec',
        np.sin(sim['dec']),
        usemask=False,
    )
config = mla.generate_default_config([
    mla.threeml.data_handlers.ThreeMLDataHandler,
    mla.PointSource,
    mla.SpatialTermFactory,
    mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory,
    mla.TimeTermFactory,
    mla.LLHTestStatisticFactory
])
config['PointSource']['name'] = 'TXS0506+056'
config['PointSource']['ra'] = mla.ra_to_rad(5, 9, 25.9645434784)
config['PointSource']['dec'] = mla.dec_to_rad(1, 5, 41, 35.333636817)
config['ThreeMLDataHandler']['dec_bandwidth (rad)'] = np.deg2rad(3)
config['ThreeMLDataHandler']['dec_cut_location']=mla.dec_to_rad(1, 5, 41, 35.333636817)
source = mla.PointSource(config['PointSource'])

data_handler_79 = mla.threeml.data_handlers.ThreeMLDataHandler(
    config['ThreeMLDataHandler'], sim, (data, grl),background_time_profile=background_time_profile,signal_time_profile=inject_signal_time_profile)
spatial_term_factory_79 = mla.SpatialTermFactory(config['SpatialTermFactory'], data_handler_79, source)
energy_term_factory_79 = mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory(config['ThreeMLPSEnergyTermFactory'], data_handler_79,source)
time_term_factory_79 = mla.TimeTermFactory(config['TimeTermFactory'],background_time_profile,inject_signal_time_profile)
llh_factory_79 = mla.LLHTestStatisticFactory(config['LLHTestStatisticFactory'],[spatial_term_factory_79,energy_term_factory_79,time_term_factory_79])
icecube_79=IceCubeLike("txs_79",data,data_handler_79,llh_factory_79,source,verbose=False)


WARNING RuntimeWarning: invalid value encountered in true_divide



In [4]:
#Loading all the data , MC and good run list
DATA_PATH = "/data/i3store/users/mjlarson/ps_tracks/version-004-p00"

# Read in all of the MC files 
sim_files = DATA_PATH + "/IC59*MC*npy"
sim = np.load([i for i in glob.glob(sim_files)][0])
#n_keep = int(0.5*len(sim)) #server don't have that much memory for me to 
#sim = np.random.choice(sim, n_keep)
sim=rf.append_fields(sim, 'sindec',
                     np.sin(sim['dec']),
                     usemask=False)


# Read in all of the data files
data_files = DATA_PATH + "/IC59_*exp.npy"
listofdata = []
data = read([i for i in glob.glob(data_files)])


# Set the angular error floor to 0.2 degrees
#data['angErr'][data['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
sim['angErr'][sim['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
np.random.seed(3) #for reproduce
data['ra'] = np.random.uniform(0, 2*np.pi, size=len(data))
grlfile = DATA_PATH + "/GRL/IC59*_exp.npy"
grl = read([i for i in glob.glob(grlfile)])
livetime = np.sum(grl['livetime'])
bkg_days = grl['stop'][-1]-grl['start'][0]
background_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
inject_signal_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
if 'sindec' not in data.dtype.names:
    data = rf.append_fields(
        data,
        'sindec',
        np.sin(data['dec']),
        usemask=False,
    )
if 'sindec' not in sim.dtype.names:
    sim = rf.append_fields(
        sim,
        'sindec',
        np.sin(sim['dec']),
        usemask=False,
    )
config = mla.generate_default_config([
    mla.threeml.data_handlers.ThreeMLDataHandler,
    mla.PointSource,
    mla.SpatialTermFactory,
    mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory,
    mla.TimeTermFactory,
    mla.LLHTestStatisticFactory
])
config['PointSource']['name'] = 'TXS0506+056'
config['PointSource']['ra'] = mla.ra_to_rad(5, 9, 25.9645434784)
config['PointSource']['dec'] = mla.dec_to_rad(1, 5, 41, 35.333636817)
config['ThreeMLDataHandler']['dec_bandwidth (rad)'] = np.deg2rad(3)
config['ThreeMLDataHandler']['dec_cut_location']=mla.dec_to_rad(1, 5, 41, 35.333636817)
source = mla.PointSource(config['PointSource'])

data_handler_59 = mla.threeml.data_handlers.ThreeMLDataHandler(
    config['ThreeMLDataHandler'], sim, (data, grl),background_time_profile=background_time_profile,signal_time_profile=inject_signal_time_profile)
spatial_term_factory_59 = mla.SpatialTermFactory(config['SpatialTermFactory'], data_handler_59, source)
energy_term_factory_59 = mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory(config['ThreeMLPSEnergyTermFactory'], data_handler_59,source)
time_term_factory_59 = mla.TimeTermFactory(config['TimeTermFactory'],background_time_profile,inject_signal_time_profile)
llh_factory_59 = mla.LLHTestStatisticFactory(config['LLHTestStatisticFactory'],[spatial_term_factory_59,energy_term_factory_59,time_term_factory_59])
icecube_59=IceCubeLike("txs_59",data,data_handler_59,llh_factory_59,source,verbose=False)


WARNING RuntimeWarning: invalid value encountered in true_divide



In [5]:
#Loading all the data , MC and good run list
DATA_PATH = "/data/i3store/users/mjlarson/ps_tracks/version-004-p00"

# Read in all of the MC files 
sim_files = DATA_PATH + "/IC40*MC*npy"
sim = np.load([i for i in glob.glob(sim_files)][0])
#n_keep = int(0.5*len(sim)) #server don't have that much memory for me to 
#sim = np.random.choice(sim, n_keep)
sim=rf.append_fields(sim, 'sindec',
                     np.sin(sim['dec']),
                     usemask=False)


# Read in all of the data files
data_files = DATA_PATH + "/IC40_*exp.npy"
listofdata = []
data = read([i for i in glob.glob(data_files)])


# Set the angular error floor to 0.2 degrees
#data['angErr'][data['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
sim['angErr'][sim['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
np.random.seed(2) #for reproduce
data['ra'] = np.random.uniform(0, 2*np.pi, size=len(data))
grlfile = DATA_PATH + "/GRL/IC40*_exp.npy"
grl = read([i for i in glob.glob(grlfile)])
livetime = np.sum(grl['livetime'])
bkg_days = grl['stop'][-1]-grl['start'][0]
background_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
inject_signal_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
if 'sindec' not in data.dtype.names:
    data = rf.append_fields(
        data,
        'sindec',
        np.sin(data['dec']),
        usemask=False,
    )
if 'sindec' not in sim.dtype.names:
    sim = rf.append_fields(
        sim,
        'sindec',
        np.sin(sim['dec']),
        usemask=False,
    )
config = mla.generate_default_config([
    mla.threeml.data_handlers.ThreeMLDataHandler,
    mla.PointSource,
    mla.SpatialTermFactory,
    mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory,
    mla.TimeTermFactory,
    mla.LLHTestStatisticFactory
])
config['PointSource']['name'] = 'TXS0506+056'
config['PointSource']['ra'] = mla.ra_to_rad(5, 9, 25.9645434784)
config['PointSource']['dec'] = mla.dec_to_rad(1, 5, 41, 35.333636817)
config['ThreeMLDataHandler']['dec_bandwidth (rad)'] = np.deg2rad(3)
config['ThreeMLDataHandler']['dec_cut_location']=mla.dec_to_rad(1, 5, 41, 35.333636817)
source = mla.PointSource(config['PointSource'])

data_handler_40 = mla.threeml.data_handlers.ThreeMLDataHandler(
    config['ThreeMLDataHandler'], sim, (data, grl),background_time_profile=background_time_profile,signal_time_profile=inject_signal_time_profile)
spatial_term_factory_40 = mla.SpatialTermFactory(config['SpatialTermFactory'], data_handler_40, source)
energy_term_factory_40 = mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory(config['ThreeMLPSEnergyTermFactory'], data_handler_40,source)
time_term_factory_40 = mla.TimeTermFactory(config['TimeTermFactory'],background_time_profile,inject_signal_time_profile)
llh_factory_40 = mla.LLHTestStatisticFactory(config['LLHTestStatisticFactory'],[spatial_term_factory_40,energy_term_factory_40,time_term_factory_40])
icecube_40=IceCubeLike("txs_40",data,data_handler_40,llh_factory_40,source,verbose=False)


WARNING RuntimeWarning: invalid value encountered in true_divide



In [6]:
#Loading all the data , MC and good run list
DATA_PATH = "/data/i3store/users/mjlarson/ps_tracks/version-004-p00"

# Read in all of the MC files 
sim_files = DATA_PATH + "/IC59*MC*npy"
sim = np.load([i for i in glob.glob(sim_files)][0])
#n_keep = int(0.5*len(sim)) #server don't have that much memory for me to 
#sim = np.random.choice(sim, n_keep)
sim=rf.append_fields(sim, 'sindec',
                     np.sin(sim['dec']),
                     usemask=False)


# Read in all of the data files
data_files = DATA_PATH + "/IC59_*exp.npy"
listofdata = []
data = read([i for i in glob.glob(data_files)])


# Set the angular error floor to 0.2 degrees
#data['angErr'][data['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
sim['angErr'][sim['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
#np.random.seed(3) #for reproduce
data['ra'] = np.random.uniform(0, 2*np.pi, size=len(data))
grlfile = DATA_PATH + "/GRL/IC59*_exp.npy"
grl = read([i for i in glob.glob(grlfile)])
livetime = np.sum(grl['livetime'])
bkg_days = grl['stop'][-1]-grl['start'][0]
background_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
inject_signal_time_profile = mla.time_profiles.UniformProfile({'start':grl['start'][0], 'length':bkg_days})
if 'sindec' not in data.dtype.names:
    data = rf.append_fields(
        data,
        'sindec',
        np.sin(data['dec']),
        usemask=False,
    )
if 'sindec' not in sim.dtype.names:
    sim = rf.append_fields(
        sim,
        'sindec',
        np.sin(sim['dec']),
        usemask=False,
    )
config = mla.generate_default_config([
    mla.threeml.data_handlers.ThreeMLDataHandler,
    mla.PointSource,
    mla.SpatialTermFactory,
    mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory,
    mla.TimeTermFactory,
    mla.LLHTestStatisticFactory
])
config['PointSource']['name'] = 'TXS0506+056'
config['PointSource']['ra'] = mla.ra_to_rad(5, 9, 25.9645434784)
config['PointSource']['dec'] = mla.dec_to_rad(1, 5, 41, 35.333636817)
config['ThreeMLDataHandler']['dec_bandwidth (rad)'] = np.deg2rad(3)
config['ThreeMLDataHandler']['dec_cut_location']=mla.dec_to_rad(1, 5, 41, 35.333636817)
source = mla.PointSource(config['PointSource'])

data_handler_59 = mla.threeml.data_handlers.ThreeMLDataHandler(
    config['ThreeMLDataHandler'], sim, (data, grl),background_time_profile=background_time_profile,signal_time_profile=inject_signal_time_profile)
spatial_term_factory_59 = mla.SpatialTermFactory(config['SpatialTermFactory'], data_handler_59, source)
energy_term_factory_59 = mla.threeml.sob_terms.ThreeMLPSEnergyTermFactory(config['ThreeMLPSEnergyTermFactory'], data_handler_59,source)
time_term_factory_59 = mla.TimeTermFactory(config['TimeTermFactory'],background_time_profile,inject_signal_time_profile)
llh_factory_59 = mla.LLHTestStatisticFactory(config['LLHTestStatisticFactory'],[spatial_term_factory_59,energy_term_factory_59,time_term_factory_59])
icecube_59=IceCubeLike("txs_59",data,data_handler_59,llh_factory_59,source,verbose=False)


WARNING RuntimeWarning: invalid value encountered in true_divide



In [7]:
TXS_sp = astromodels.Powerlaw(piv=100e3) #In GeV.Setting a pivot energy is important
#otherwise the minimizer will not find the current flux norm and spectral index minimum(as they are correlated)
TXS_sp.K.bounds = (1e-50,1e-15)
TXS_sp.index.bounds = (-4,-1)
TXS_sp.K = 1e-19
TXS_sp.index = -2
TXS = mla.threeml.IceCubeLike.NeutrinoPointSource("TXS", ra=77.3583, dec=5.6931, spectral_shape=TXS_sp)
model = astromodels.Model(TXS)

[WARNING ] We have set the min_value of Powerlaw.K to 1e-99 because there was a postive transform
[WARNING ] The current value of the parameter K (1.0) was above the new maximum 1e-15.


In [8]:
import warnings
warnings.filterwarnings("ignore")
icecube.verbose=False
icecube_79.verbose=False
icecube_59.verbose=False
icecube_40.verbose=False
IceCubedata = threeML.DataList(icecube,icecube_79,icecube_59,icecube_40)
#IceCubedata = threeML.DataList(icecube_59)
jl = threeML.JointLikelihood(model, IceCubedata)
jl.set_minimizer("minuit");




[INFO    ] set the minimizer to minuit
[INFO    ] set the minimizer to MINUIT


In [9]:
import time
#data['ra'] = np.random.uniform(0, 2*np.pi, size=len(data))
#icecube_59.update_data(data)
start_time = time.time()
jl.fit(quiet=True) #take a while to run, it is 3 years of data. Sometime the minimizer will fail.
print("--- %s seconds ---" % (time.time() - start_time))

[WARNING ] 96.7 percent of samples have been thrown away because they failed the constraints on the parameters. This results might not be suitable for error propagation. Enlarge the boundaries until you loose less than 1 percent of the samples.
[WARNING ] The current value of the parameter K (1.0) was above the new maximum 1e-15.
--- 47.14252424240112 seconds ---


In [10]:
jl.current_minimum

-3.0958490601021814e-05

In [11]:
jl.results.display()

Best fit values:



,result,unit
parameter,,
TXS.spectrum.main.Powerlaw.K,(3.3 -3.3 +7) x 10^23,1 / (cm2 keV s)
TXS.spectrum.main.Powerlaw.index,-4 +/- 9,



Correlation matrix:



1.00,0.34
0.34,1.00



Values of -log(likelihood) at the minimum:



,-log(likelihood)
total,-0.000031
txs,0.006715
txs_40,0.000336
txs_59,-0.002191
txs_79,-0.004891



Values of statistical measures:



,statistical measures
AIC,3.999947
BIC,28.261176


In [12]:
analysislist = mla.threeml.IceCubeLike.icecube_analysis([icecube,icecube_79,icecube_59,icecube_40])

In [19]:
analysislist.injection(n_signal=30,poisson=True)
TXS_sp = astromodels.Powerlaw(piv=100e3) #In GeV.Setting a pivot energy is important
#otherwise the minimizer will not find the current flux norm and spectral index minimum(as they are correlated)
TXS_sp.K.bounds = (1e-50,1e-15)
TXS_sp.index.bounds = (-4,-1)
TXS_sp.K = 1e-19
TXS_sp.index = -2
TXS =  mla.threeml.IceCubeLike.NeutrinoPointSource("TXS", ra=77.3583, dec=5.6931, spectral_shape=TXS_sp)
model = astromodels.Model(TXS)
import warnings
warnings.filterwarnings("ignore")
icecube.verbose=False
icecube_79.verbose=False
icecube_59.verbose=False
icecube_40.verbose=False
IceCubedata = threeML.DataList(icecube,icecube_79,icecube_59,icecube_40)
#IceCubedata = threeML.DataList(icecube_59)
jl = threeML.JointLikelihood(model, IceCubedata)
jl.set_minimizer("minuit");



[WARNING ] We have set the min_value of Powerlaw.K to 1e-99 because there was a postive transform
[WARNING ] The current value of the parameter K (1.0) was above the new maximum 1e-15.
[INFO    ] set the minimizer to minuit
[INFO    ] set the minimizer to MINUIT


In [20]:
start_time = time.time()
jl.fit(quiet=True) 
print("--- %s seconds ---" % (time.time() - start_time))

[WARNING ] The current value of the parameter K (1.0) was above the new maximum 1e-15.
--- 46.93133854866028 seconds ---


In [21]:
jl.results.display()

Best fit values:



,result,unit
parameter,,
TXS.spectrum.main.Powerlaw.K,(2.7 -0.4 +0.5) x 10^-25,1 / (cm2 keV s)
TXS.spectrum.main.Powerlaw.index,-2.19 +/- 0.11,



Correlation matrix:



1.00,0.13
0.13,1.00



Values of -log(likelihood) at the minimum:



,-log(likelihood)
total,-52.781682
txs,-43.320410
txs_40,-6.843299
txs_59,0.206401
txs_79,-2.824374



Values of statistical measures:



,statistical measures
AIC,-101.563356
BIC,-77.302733
